<a href="https://colab.research.google.com/github/Tom-Lennox/Kaggle/blob/master/titanic_20200410.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# title

In [2]:
# kaggle APIセット
!pip install kaggle

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)
# 「Download 100%.」と表示で成功。

# [kaggle.json]を持参する。
from google.colab import drive
drive.mount('/content/drive')

# jsonファイルを指定の場所に配置
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
# titanicをダウンロードします。
!kaggle competitions download -c titanic

# jsonファイルを指定の場所に配置
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
# titanicをダウンロードします。
!kaggle competitions download -c titanic
!ls

Download 100%.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 22.8MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 3.13MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 27.9MB/s]
test.csv: Skipping, found more recently modified local copy (use --force to force download)
gender_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
train.csv: Skipping, found more recently modified local copy (use --force to force download)
adc.json  drive  gender_submission.csv	sample_data  test.csv  train.csv


In [4]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
# RandomForestClassifier
# 複数の決定木を使って各決定木の予測結果の多数決で結果を求める方法
# ｜Decision Tree(決定木
# 　｜Classification Tree 分類木
# 　｜Regression Tree 回帰木

train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")
# とりあえずファイルを読み込む

# train.head()
# 名前確かに敬称あるね。
# ticketはアルファベットがヒントになりそう。
# Name: 敬称を別カラムに移植、削除
# objectで抽出したものをone-hot-encordingで置換（手動置換は無駄。
# ticket ⇒ 条件でカラム作成して代入 ⇒ 

print('▼　inull list')
print(train.isnull().sum()[train.isnull().sum() > 0].index.tolist())
# null抽出

print('▼　Embarked fillna')
# Embarked　処理：
# １．noneで置き換えてohe
# ２．mean()
train.head()
train['Embarked'] = train['Embarked'].fillna("None")
print(train['Embarked'].unique())

print('▼ Ticket')
train.head()
print(train['Ticket'].unique().tolist())

# 【統合先】

▼　inull list
['Age', 'Cabin', 'Embarked']
▼　Embarked fillna
['S' 'C' 'Q' 'None']
▼ Ticket
['A/5 21171', 'PC 17599', 'STON/O2. 3101282', '113803', '373450', '330877', '17463', '349909', '347742', '237736', 'PP 9549', '113783', 'A/5. 2151', '347082', '350406', '248706', '382652', '244373', '345763', '2649', '239865', '248698', '330923', '113788', '347077', '2631', '19950', '330959', '349216', 'PC 17601', 'PC 17569', '335677', 'C.A. 24579', 'PC 17604', '113789', '2677', 'A./5. 2152', '345764', '2651', '7546', '11668', '349253', 'SC/Paris 2123', '330958', 'S.C./A.4. 23567', '370371', '14311', '2662', '349237', '3101295', 'A/4. 39886', 'PC 17572', '2926', '113509', '19947', 'C.A. 31026', '2697', 'C.A. 34651', 'CA 2144', '2669', '113572', '36973', '347088', 'PC 17605', '2661', 'C.A. 29395', 'S.P. 3464', '3101281', '315151', 'C.A. 33111', 'S.O.C. 14879', '2680', '1601', '348123', '349208', '374746', '248738', '364516', '345767', '345779', '330932', '113059', 'SO/C 14885', '3101278', 'W./C. 66

In [17]:
train['Age'].fillna(train.Age.mean(), inplace=True)

# Embarkedは最後に
# ▼
# null
print(train.isnull().sum()[train.isnull().sum() > 0].index.tolist())
# ▲
combine1 = [train]
print(combine1)
for i in train:
  print(i)
  # 12

for train in combine1:
  # print(train.Name)
  # print(train.Name.str.extract(' ([A-Za-z]+).', expand=False).tolist())
  # 一文字以上のアルファベット+「.」のパターンを抽出
  train['Salutation'] = train.Name.str.extract(' ([A-Za-z]+).', expand=False) 

['Cabin']
[     PassengerId  Survived  Pclass  ... Cabin Embarked  Salutation
0              1         0       3  ...   NaN        S          Mr
1              2         1       1  ...   C85        C         Mrs
2              3         1       3  ...   NaN        S        Miss
3              4         1       1  ...  C123        S         Mrs
4              5         0       3  ...   NaN        S          Mr
..           ...       ...     ...  ...   ...      ...         ...
886          887         0       2  ...   NaN        S         Rev
887          888         1       1  ...   B42        S        Miss
888          889         0       3  ...   NaN        S        Miss
889          890         1       1  ...  C148        C          Mr
890          891         0       3  ...   NaN        Q          Mr

[891 rows x 13 columns]]
PassengerId
Survived
Pclass
Name
Sex
Age
SibSp
Parch
Ticket
Fare
Cabin
Embarked
Salutation


In [0]:
# 追記
# ▼
# 処理のために結合
train_x = pd.read_csv("/content/train.csv")
test_x = pd.read_csv("/content/test.csv")
print('train_x: ' + str(train_x.shape))
print('test_x: '  + str(test_x.shape))
# sort: カラム名のsort。
all_data = pd.concat([train_x,test_x],axis=0,sort=True)
print('all_data: '  + str(all_data.shape))
# ▲
# ▼
# # 学習データとテストデータに再分割
# [tips]shape使え。
# train_x = all_data.iloc[:train_x.shape[0],:].reset_index(drop=True)
# test_x = all_data.iloc[train_x.shape[0]:,:].reset_index(drop=True)
# ▲
# ▼
# nullは下記で取得できる。
print('▼train_x')
print(train_x.isnull().sum())
print('▼test_x')
print(test_x.isnull().sum())
print(train[['Age','Cabin','Embarked']])
all_data.head(3)
# ▲
# ▼
# pandas, one-hot-encodingを行う。
one-hot-encoding
cal_list = all_data.dtypes[all_data.dtypes=="object"].index.tolist()
all_data = pd.get_dummies(all_data,columns=cal_list)
all_data
# ▲

train_x: (891, 12)
test_x: (418, 11)
all_data: (1309, 12)
▼train_x
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
▼test_x
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64
      Age Cabin Embarked
0    22.0   NaN        S
1    38.0   C85        C
2    26.0   NaN        S
3    35.0  C123        S
4    35.0   NaN        S
..    ...   ...      ...
886  27.0   NaN        S
887  19.0   B42        S
888   NaN   NaN        S
889  26.0  C148        C
890  32.0   NaN        Q

[891 rows x 3 columns]


,Age,Fare,Parch,PassengerId,Pclass,SibSp,Survived,Cabin_A10,Cabin_A11,Cabin_A14,Cabin_A16,Cabin_A18,Cabin_A19,Cabin_A20,Cabin_A21,Cabin_A23,Cabin_A24,Cabin_A26,Cabin_A29,Cabin_A31,Cabin_A32,Cabin_A34,Cabin_A36,Cabin_A5,Cabin_A6,Cabin_A7,Cabin_A9,Cabin_B10,Cabin_B101,Cabin_B102,Cabin_B11,Cabin_B18,Cabin_B19,Cabin_B20,Cabin_B22,Cabin_B24,Cabin_B26,Cabin_B28,Cabin_B3,Cabin_B30,...,Ticket_SOTON/OQ 3101316,Ticket_SOTON/OQ 3101317,Ticket_SOTON/OQ 392076,Ticket_SOTON/OQ 392082,Ticket_SOTON/OQ 392083,Ticket_SOTON/OQ 392086,Ticket_SOTON/OQ 392089,Ticket_SOTON/OQ 392090,Ticket_STON/O 2. 3101268,Ticket_STON/O 2. 3101269,Ticket_STON/O 2. 3101273,Ticket_STON/O 2. 3101274,Ticket_STON/O 2. 3101275,Ticket_STON/O 2. 3101280,Ticket_STON/O 2. 3101285,Ticket_STON/O 2. 3101286,Ticket_STON/O 2. 3101288,Ticket_STON/O 2. 3101289,Ticket_STON/O 2. 3101291,Ticket_STON/O 2. 3101292,Ticket_STON/O 2. 3101293,Ticket_STON/O 2. 3101294,Ticket_STON/O2. 3101270,Ticket_STON/O2. 3101271,Ticket_STON/O2. 3101279,Ticket_STON/O2. 3101282,Ticket_STON/O2. 3101283,Ticket_STON/O2. 3101290,Ticket_STON/OQ. 369943,Ticket_SW/PP 751,Ticket_W./C. 14258,Ticket_W./C. 14260,Ticket_W./C. 14263,Ticket_W./C. 14266,Ticket_W./C. 6607,Ticket_W./C. 6608,Ticket_W./C. 6609,Ticket_W.E.P. 5734,Ticket_W/C 14208,Ticket_WE/P 5735
0,22.0,7.2500,0,1,3,1,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,38.0,71.2833,0,2,1,1,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,26.0,7.9250,0,3,3,0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,35.0,53.1000,0,4,1,1,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,35.0,8.0500,0,5,3,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,NaN,8.0500,0,1305,3,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
414,39.0,108.9000,0,1306,1,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
415,38.5,7.2500,0,1307,3,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
416,NaN,8.0500,0,1308,3,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
